In [3]:
model_name = 'baseline_sparse'
data_dir = './data/mlboot_dataset/'
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp

In [ ]:
mat3 = sp.load_npz(data_dir+'mat1.npz').tolil()

In [ ]:
df = pd.read_csv(data_dir + 'preprocessed.csv') 
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')
df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

In [ ]:
mat3.tocsr()[df_train_index.tolist()].max()

In [2]:
df = pd.read_csv(data_dir + 'preprocessed.csv') 
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

mat1 = sp.load_npz(data_dir+'dmat1.npz').tolil()
mat2 = sp.load_npz(data_dir+'dmat2.npz').tolil()
mat3 = sp.load_npz(data_dir+'dmat3.npz').tolil()

In [3]:
df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [4]:
%%time
train_mat1 = mat1[df_train_index]

CPU times: user 12.3 s, sys: 546 ms, total: 12.8 s
Wall time: 12.8 s


In [5]:
test_mat1 = mat1[df_test_index]
train_mat2 = mat2[df_train_index]
test_mat2 = mat2[df_test_index]
train_mat3 = mat3[df_train_index]
test_mat3 = mat3[df_test_index]

In [15]:
train_cols = [u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    'max_depth' : 12,
    #'min_data' : 30,
    #'lambda_l2' : 15,
    'min_sum_hessian_in_leaf' : 0.3,
    'lambda_l1' : 2.5,
    #'max_drop' : 1,
    'learning_rate': 0.01,
    'feature_fraction': 0.7,
    'verbose': 0
}

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

#y_pred = 0
#y_oof = X[['uid','target']].copy()
#y_oof['target'] = np.nan

for train_index,test_index in kf.split(X):
    if ifold < 2:
        ifold += 1
        continue
        
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, train_mat1[train_index], train_mat2[train_index], train_mat3[train_index]
    ]).tocsr()
    print('prepare valid')
    X_va = sp.hstack([
        X_va, train_mat1[test_index], train_mat2[test_index], train_mat3[test_index]
    ]).tocsr()
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = sp.hstack([
        X_te, test_mat1, test_mat2, test_mat3
    ]).tocsr()   
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

('fold', 2)
prepare train
prepare valid
[50]	valid_0's auc: 0.634717
[100]	valid_0's auc: 0.63679
[150]	valid_0's auc: 0.641467
[200]	valid_0's auc: 0.646252
[250]	valid_0's auc: 0.64982
[300]	valid_0's auc: 0.652323
[350]	valid_0's auc: 0.65598
[400]	valid_0's auc: 0.658425
[450]	valid_0's auc: 0.660575
[500]	valid_0's auc: 0.661853
[550]	valid_0's auc: 0.663228
[600]	valid_0's auc: 0.664251
[650]	valid_0's auc: 0.665143
[700]	valid_0's auc: 0.66594
[750]	valid_0's auc: 0.666026
[800]	valid_0's auc: 0.666771
[850]	valid_0's auc: 0.6673
[900]	valid_0's auc: 0.667427
[950]	valid_0's auc: 0.668013
[1000]	valid_0's auc: 0.668522
[1050]	valid_0's auc: 0.668425
[1100]	valid_0's auc: 0.668681
[1150]	valid_0's auc: 0.668672
[1200]	valid_0's auc: 0.668876
[1250]	valid_0's auc: 0.668886
[1300]	valid_0's auc: 0.668674
[1350]	valid_0's auc: 0.668513
[1400]	valid_0's auc: 0.668451
[1450]	valid_0's auc: 0.668373
Early stopping, best iteration is:
[1271]	valid_0's auc: 0.668977
(2, 0.668976510596361

[1500]	valid_0's auc: 0.687666
[1550]	valid_0's auc: 0.687563
[1600]	valid_0's auc: 0.687395
Early stopping, best iteration is:
[1406]	valid_0's auc: 0.688223
(9, 0.6882229656723633)
prepare test


In [19]:
np.mean([0.6964301536442619, 0.6795403698006517, 0.6689765105963613, 0.6875355790327728, 0.6773371572837631,
0.6817851725307851, 0.678971051709, 0.688371129105999, 0.6794184751640736, 0.6882229656723633])

0.6826588564540031

In [16]:
model_name = 'baseline_sparse_10folds'

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

In [17]:
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub.head()

('isnull?', False)


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.110928
1,ac4b8244f3ae82df511b002257473c11,0.060637
2,483d8b91e49522c8a5bbe37f3872c749,0.091964
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.037642
4,fdbfba9842ff0bf86d600eb334c7c42b,0.035158


In [18]:
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)